---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

In [1]:
import re
from time import sleep

import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

medical_dates_df = pd.Series(doc)

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates.

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:

import itertools
import copy
import typing
from functools import lru_cache
from string import digits
from typing import Pattern
import datetime
import re
import numpy as np


In [ ]:
class DateFinderx:
    """
    The purpose of this class is to find dates in text strings.
    :param raw_text: Raw text from which a date is to be extracted.
    :
    """

    def __repr__(self):
        """
        :return: Date in american mm/dd/yyyy format (all numeric)
        """
        return f"{self.month}/{self.day}/{self.year}"

    def __init__(self, raw_text: str = ''):
        """


        """
        self.month = None
        self.day = None
        self.year = None

        cleaned_text = self.clean_text(raw_text=raw_text)
        result_dict = self.apply_regexes(search_text=cleaned_text)
        self.pydate = self.create_pydate(result_dict=result_dict)

    def clean_text(self, raw_text: str = None) -> str:
        """
        The purpose of this function is to aid regex effectiveness by cleaning out character (period, comma, colon,
        semicolon) from the test string. Double spaces are also eliminated.

        :param raw_text: A string which will be used for date analysis.
        """
        clean_text = re.sub(pattern=r"[.,;:]", string=raw_text, repl='')
        cleaner_text = re.sub(pattern=r"\s{2,}", string=clean_text, repl=' ')
        return cleaner_text

    def apply_regexes(self, search_text: str = str()):
        """
        This function applies the regex patterns to the string, iterating until a match is found.
        :param search_text: String to search for a date with regexes It outputs a group dictionary
        :return: dictionary of month, day and year
        """
        date_capture_regex = DateCaptureRegex.create_date_regex()

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Iterate through the regexes. Have three cases, first, no match, just continue iterating
        # second case, one result, grab the group dictionary of the first.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        for dcr in date_capture_regex:
            results = tuple(re.finditer(pattern=dcr, string=search_text))
            if len(results) == 0:
                continue
            if len(results) == 1:
                return results[0].groupdict()
            if len(results) > 1:
                validity_results = [(result, NicDate().valid_date(in_match_result=result)) for result in results]
                validity_results = sorted(validity_results, key=lambda x: x[1], reverse=True)
                best_result = validity_results[0][0].groupdict()
                return best_result

    def create_pydate(self, result_dict: dict = None):
        """
        The purpose of this function is to take the results from regex, delivered in a dictionary format and
        produce a python date object.
        :param result_dict: group dictionary of the date regex result.
        :return: Python Date object representing the values of the dictionary.
        """

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # In the first section here, we get the year result. The year can be wither two or four digits.
        # We extract the year as an integer. Two digit will yield a result less than 100. In which case
        # (According to the requirements of the project we assume all years are 19XX) we just add 1900.
        # Otherwise, we pass along the year as an integer.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        year_seed = int(result_dict.get('year'))
        if year_seed < 100:
            year_seed = year_seed + 1900
        self.year = year_seed

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Now we extract the month. According to the specs, if the month is not present (None in Regex)
        # Then we assume it's the first month of the year. Month can come in as a name "June" or a number
        # in string format form the regex.  If it's a name, we use the month conversion dictionary
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        month_seed = result_dict.get('month', '1').lower()
        if month_seed.isalpha():
            month_num = int(NicDate.month_conversion_dict().get(month_seed, None))
        else:
            month_num = int(month_seed)
            if not NicDate().is_valid_month(month=month_num):
                month_num = 1

        self.month = month_num

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #  Extract the day. Day should always come in as numeric; so no high processing needed.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        self.day = int(result_dict.get('day', 1))

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #  create a datetime.date object with the info.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        try:
            return datetime.date(year=self.year, month=self.month, day=self.day)
        except ValueError as uhoh:
            print(f"{uhoh}")


In [ ]:
from dataclasses import dataclass
from functools import lru_cache
import numpy as np


@dataclass
class NicDate(dict):
    """
    The NicDate Class inputs the string values from regex search for month, day and year.
    It inherits from dictionary.
    A dataclass is used instead of a raw dictionary, because a dictionary is not hashable, and can not be cached by
    @lru_cache. This class is  hashable and able to be reused in @lru_cache functions.

    :raises: Value Error if verify_not_none method is called and
    """

    def __hash__(self):
        # Define a specific has for this class, so it can be cached.
        hash_seed = hash(f"{self.month}/{self.date}/{self.year}")
        return hash_seed

    def __init__(self, month: str = str(), date: str = str(), year: str = ()):
        super().__init__()
        self.month = month
        self.date = date
        self.year = year

    def __repr__(self):
        return f"Day = {self.date} Month = {self.month} Year = {self.year}"

    def verify_not_none(self):
        """
        This function returns an error when called if any of the components are none.

        :raises ValueError if any of the components are None.
        """
        if any((self.month is None, self.date is None, self.year is None)):
            raise ValueError("None Value passed")

    @staticmethod
    @lru_cache(maxsize=12)
    def find_largest_day(result_month: int = 0) -> dict:
        months_numerical = np.arange(start=1, stop=13).astype(np.uint8)
        month_max_days = np.array((31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)).astype(np.uint8)
        largest_day_dict = {a: b for a, b in zip(months_numerical, month_max_days)}
        largest_day = largest_day_dict.get(result_month, np.nan)
        return largest_day


In [ ]:
def date_sorter():
    
     # ~~~~~~~~~~~~~~~~~~~~~~~~
    # read in the source file
    # ~~~~~~~~~~~~~~~~~~~~~~~~
    doc = []
    with open(r'.\dates.txt') as file:
        for line in file:
            doc.append(line)

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Run the primary class and create a tuple where the first number is sequential and the 2nd number is text
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    pre_sorted_results = []
    for line_counter, line_text in enumerate(doc):
        try:
            pre_sorted_results.append((line_counter, DateFinder(raw_string=line_text).convert_to_python_datetime()))
        except ValueError as uhoh:
            print(f"{uhoh}")

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # sort on the date to get the value
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    sorted_results = copy.deepcopy(pre_sorted_results)
    sorted_results.sort(key=lambda x: x[1])
    final_results = [x[0] for x in sorted_results]


    return final_results